# Random Forest
### From scratch

## Imports

In [16]:
from random import randrange
from csv import reader
from math import sqrt

## 1. Gini Index

In [17]:
%%capture
%run DecisionTree.ipynb

## 2. Create split

In [18]:
# Select the best split point for a dataset with n random features
def get_split(dataset, n_features):
    classes = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, classes)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create child splits for a node or make terminal (typical recursive tree process)
def Split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # if left and right are leaves or max depth is reached, terminal
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        Split(node['left'], max_depth, min_size, n_features, depth+1)
    
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        Split(node['right'], max_depth, min_size, n_features, depth+1)

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

## 3. Build a decision tree

In [4]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    Split(root, max_depth, min_size, n_features, 1)
    return root
 
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict): return predict(node['left'], row)
        else: return node['left']
    else:
        if isinstance(node['right'], dict): return predict(node['right'], row)
        else: return node['right']


## 4. Build a random forest

In [34]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample
 
# Make a prediction with a list of bagged trees (return mode)
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)
 
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

## Testing methods

In [35]:
x = [1,2,3,4,5,6,7]
sx = set(x)
sx
x.count


<function list.count(value, /)>

In [36]:
%%capture
%run ../NaiveBayes/NaiveBayes_fs.ipynb
%run ../DecisionTree/DecisionTree_fs.ipynb

## Test

#### Load data

In [37]:
filename = 'D:/data/csv/sonar-alldata.csv'
dataset = load_csv(filename)
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
str_column_to_int(dataset, len(dataset[0])-1)
#dataset

{'M': 0, 'R': 1}

#### Evaluate

In [38]:
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10, 15]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{1}
1
{0}
0
{1}
1
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{1}
1
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{1}
1
{1}
1
{0}
0
{1}
1
{1}
1
{0}
0
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{0}
0
{0}
0
{1}
1
{0}
0
{1}
1
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{0}
0
{1}
1
{1}
1
{0}
0
{1}
1
{1}
1
{0}
0
{0}
0
{1}
1
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{1}
1
{1}
1
{0}
0
{1}
1
{1}
1
{1}
1
{1}
1
{1}
1
{0}
0
{1}
1
{0}
0
{0}
0
{0}
